### Naludaq Version
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Prerequisites
+ Acquisition captured previously (can be from NaluScope)

In [ ]:
# Run this code for plotting
import matplotlib.pyplot as plt

from naluexamples.helpers.plotting import get_color_mapping, set_plot_style

def plot_event(event: dict, channels: list[int]=None, title: str=None):
    """Plot an event dict.

    Args:
        event (dict): the parsed event
        channels (list[int]): Channels to plot. Defaults to all channels.
        title (str): Optional title for the plot
    """
    channels = channels or range(len(event["data"]))
    title = title or f"Event #{event.get('pkg_num', '?')}"


    # Plot the event
    plt.figure(figsize=(8,8), constrained_layout=True)
    plt.xlabel("Time")
    plt.ylabel("ADC Counts")
    plt.title(title)
    cmap = get_color_mapping("ocean")
    set_plot_style(font_size=18, font_family="monospace")
    for i, channel in enumerate(channels):
        time = event["time"][channel]
        data = event["data"][channel]
        plt.plot(time, data, ".-", label=f"Channel {channel}", color=cmap(i / len(channels)))
    plt.legend()
    plt.show()

## Legacy Acquisitions

Acquisitions captured in the older "legacy" format are gzipped Python pickles. 
The data structure is a bit different than the newer format, and the entire acquisition must be loaded into memory.

In [ ]:
from naludaq.io import load_pickle_acquisition

ACQ_PATH = r"<PATH TO .acq FILE>"

# Load the acquisition from disk
ACQ = load_pickle_acquisition(ACQ_PATH)

print("Board model:", ACQ.params["model"])
print("Number of events:", len(ACQ))
print("Pedestals stored:", ACQ.pedestals is not None)

Accessing events is different with legacy acquisitions, as they must be manually parsed.

In [ ]:
from naludaq.parsers import get_parser

EVENT_INDEX = 0

parser = get_parser(ACQ.params)
event = parser.parse(ACQ[EVENT_INDEX])
plot_event(event)

## Opening Without NaluDaq

It is possible to open acquisitions without using NaluDaq, as the acquisition files are just gzipped Python pickles of a dictionary object.

In [1]:
import gzip
import pickle

ACQ_PATH = r"C:\Users\nalul\New Project\asocv3 visualizer data.acq"

with gzip.GzipFile(ACQ_PATH, "rb") as f:
    acq = pickle.load(f)

# Examples on how to access the data. Use acq.keys() to see all available keys.
all_events = acq["events"]
pedestals = acq["calibration"]["pedestals"]
params = acq["metadata"]["settings"]["params"]

The dictionary contains the following tree structure:

- `events` - Unparsed event dictionaries. Must be parsed using `params` before processing.
- `calibration`
  - `pedestals` - Pedestals calibration data, used for pedestals correction
- `metadata`
  - `acq_num` - Acquisition number from NaluScope
  - `created_at` - Acquisition creation timestamp
  - `file_version` - File format version
  - `info` - Information about readout configuration
    - `connection` - Board connection information
    - `readings` - Sensor readings
    - `readout_settings` - Settings used for the readout
    - `trigger_settings` - Settings used for triggering
  - `naludaq_version` - Version of `naludaq` used to capture acquisition
  - `naluscope_version` - Version of `naluscope` used to capture acquisition
  - `name` - Acquisition name from NaluScope
  - `notes` - Notes from NaluScope
  - `settings` - Board settings at the time of capture
    - `params` - Board parameters. Necessary for some operations, including parsing and pedestals correction.
    - `registers` - Board registers and values

## Pedestals Subtraction

The events stored in the acquisitions are not pedestal-subtracted. To obtain pedestals-subtracted events, the process must be performed manually on the events.

In [ ]:
from naludaq.tools.pedestals.pedestals_correcter import PedestalsCorrecter


# Fetch an event, pedestals, and params from disk
ACQ_PATH = r"<PATH TO ACQUISITION FOLDER>"
EVENT_INDEX = 0

ACQ = load_pickle_acquisition(ACQ_PATH)

event = ACQ[EVENT_INDEX]
params = ACQ.params
pedestals = ACQ.pedestals

if pedestals is None:
    print("Acquisition must have pedestals in order to correct data!")
else:
    # Apply correction (in place)
    corrector = PedestalsCorrecter(params, pedestals)
    corrected_event = corrector.run(event, correct_in_place=True)
    plot_event(corrected_event)